In [ ]:
# Define all applicable imports
import pandas as pd
import xarray as xr
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns


# Define notable variables
# Define the range of full years of data availabe in the Earthdata base_path
start_year = 2015
end_year = 2016
base_path = '../input/earthdata-ges-disc/dataset'


def generate_dates(start_year, end_year):
    """Generates a list of dates from start_year to end_year."""
    start_date = pd.Timestamp(f'{start_year}-01-01')
    end_date = pd.Timestamp(f'{end_year}-12-31')
    return pd.date_range(start=start_date, end=end_date, freq='D')

dates = generate_dates(start_year, end_year)


# # FOR DEBUGGING PURPOSES
# # Examine the dataset

# # Open a single dataset to inspect its contents
# sample_file = f'{base_path}/3B-DAY-L.MS.MRG.3IMERG.20150101-S000000-E235959.V06.nc4.SUB.nc4'
# ds = xr.open_dataset(sample_file)

# # Print the dataset to see available variables and data structure
# print(ds)
# print(f"precipitationCal: {ds['precipitationCal'].values}")
# print(f"precipitationCal_cnt: {ds['precipitationCal_cnt'].values}")
# print(f"HQprecipitation: {ds['HQprecipitation'].values}")
# print(f"HQprecipitation_cnt: {ds['HQprecipitation_cnt'].values}")

# # Close the dataset
# ds.close()

def process_nc_files(start_year, end_year, base_path):
    """Loads and processes .nc4 files into a single DataFrame."""
    all_data = []
    
    for date in dates:
        formatted_date = date.strftime('%Y%m%d')
        file_path = f'{base_path}/3B-DAY-L.MS.MRG.3IMERG.{formatted_date}-S000000-E235959.V06.nc4.SUB.nc4'
        ds = None  # Initialize ds to None

        try:
            ds = xr.open_dataset(file_path)
            # Assuming 'precipitation' is the variable name; replace as needed
            daily_precip_in_mm = ds['precipitationCal'].mean().values
            all_data.append({'date': date, 'precipitation': daily_precip_in_mm})
        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except ValueError as e:
            print(f"Error opening file: {file_path}. Error: {e}")
        finally:
            if ds:
                ds.close()

    return pd.DataFrame(all_data)

df_precipitation = process_nc_files(start_year, end_year, base_path)


# Connect to the SQLite DB
conn = sqlite3.connect('../input/hydat-db/Hydat.sqlite3')

# Define relevant station numbers
relevant_station_numbers_query = """
SELECT STATION_NUMBER
FROM STATIONS
WHERE CONTRIBUTOR_ID = 75 AND HYD_STATUS = 'A'
AND LONGITUDE BETWEEN -131.941 AND -112.897
AND LATITUDE BETWEEN 47.126 AND 57.367;
"""

relevant_station_numbers = pd.read_sql_query(relevant_station_numbers_query, conn)
list_of_relevant_station_numbers = relevant_station_numbers['STATION_NUMBER'].tolist()

# Convert list of station numbers to a format that can be used in SQL query
station_numbers_tuple = tuple(list_of_relevant_station_numbers)

# Fetch water level data for each day of the month for BC Hydro stations
day_columns = [f"LEVEL{i}" for i in range(1, 32)]  # LEVEL1, LEVEL2, ..., LEVEL31
day_columns_str = ', '.join(day_columns)
query = f"""
SELECT STATION_NUMBER, YEAR, MONTH, NO_DAYS, FULL_MONTH, {day_columns_str}
FROM DLY_LEVELS
WHERE STATION_NUMBER IN ({','.join('?' for _ in station_numbers_tuple)})
"""

df_water_levels = pd.read_sql_query(query, conn, params=station_numbers_tuple)

# Close the connection
conn.close()


# Function to calculate the monthly mean water level
def calculate_monthly_mean(row, overall_mean):
    # Extract daily water level data into a list
    daily_levels = [row[f"LEVEL{i}"] for i in range(1, 32) if f"LEVEL{i}" in row]

    # Count the number of valid days
    valid_days = sum(not pd.isna(level) for level in daily_levels)

    # If there are missing days, use mean imputation
    if valid_days < row['NO_DAYS']:
        missing_days = row['NO_DAYS'] - valid_days
        daily_levels += [overall_mean] * missing_days

    # Calculate the mean of daily levels (including imputed values)
    return np.mean(daily_levels)

# Calculate the overall mean from all available daily data
all_daily_levels = []
for i in range(1, 32):
    all_daily_levels += df_water_levels[f"LEVEL{i}"].dropna().tolist()

overall_mean = np.mean(all_daily_levels)

# Apply the function to each row
df_water_levels['calculated_monthly_mean'] = df_water_levels.apply(lambda row: calculate_monthly_mean(row, overall_mean), axis=1)


# Merge the datasets
df_water_levels['date'] = pd.to_datetime(df_water_levels[['YEAR', 'MONTH']].assign(DAY=1))
df_combined = pd.merge(df_precipitation, df_water_levels, on='date', how='left')

# Remove the individual LEVEL columns as they are no longer needed.
level_columns = [col for col in df_combined if col.startswith('LEVEL')]
columns_to_drop = level_columns + ['NO_DAYS', 'FULL_MONTH']
df_combined.drop(columns=level_columns, inplace=True)

# Use mean imputation for missing values
mean_value = df_combined['calculated_monthly_mean'].mean()
df_combined['calculated_monthly_mean'].fillna(mean_value, inplace=True)


# Visualization

%matplotlib inline

# Convert 'precipitation' to float
df_combined['precipitation'] = df_combined['precipitation'].astype(float)

# Ensure 'calculated_monthly_mean' is also a float
df_combined['calculated_monthly_mean'] = df_combined['calculated_monthly_mean'].astype(float)

# Set the aesthetics for the plots
sns.set(style="whitegrid")

# Time Series Plot for Precipitation and Water Levels
# Set the figure size and the aspect ratio to give more space on the x-axis
plt.figure(figsize=(20, 5))

# Plot the time series data
plt.plot(df_combined['date'], df_combined['precipitation'], label='Precipitation (mm)', alpha=0.7)
plt.plot(df_combined['date'], df_combined['calculated_monthly_mean'], label='Monthly Mean Water Level', color='orange', alpha=0.7)

# Set major ticks to be at the first of each month and format them to show the month and year
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

# Rotate and align the tick labels so they look better
plt.gcf().autofmt_xdate()

# Add labels, title, and legend
plt.xlabel('Date')
plt.ylabel('Measurement')
plt.title('Time Series of Precipitation and Water Levels')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Scatter Plot for Precipitation vs. Water Levels
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_combined, x='precipitation', y='calculated_monthly_mean', alpha=0.6)
plt.xlabel('Precipitation (mm)')
plt.ylabel('Monthly Mean Water Level')
plt.title('Scatter Plot of Precipitation vs. Water Levels')
plt.grid(True)
plt.tight_layout()
plt.show()

# Box Plot for Precipitation and Water Levels
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_combined[['precipitation', 'calculated_monthly_mean']])
plt.ylabel('Measurement')
plt.title('Box Plot for Precipitation and Water Levels')
plt.grid(True)
plt.tight_layout()
plt.show()


